<a href="https://colab.research.google.com/github/Valdini/Carbon-Footprint-Multi-Agent-Reinforcement-Learning/blob/master/Source_Code_MARL_&_Global_Carbon_Footprint_School_of_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
'''
This simulation models investments by 3 agents (countries) into CO2 reductions
based on game theory and Multi-Agent Reinforcement Learning (MARL). 
It complements a research paper written by Valentin Kahn for the School of AI
Contact: valentin.kahn@gmail.com

How to use:
Run the following code in your browser by
1) Signing in to your Google account
2) Click on "Open in Playground" and "Run anyway"
3) Click on the ">" icon or by pressing Ctrl + Enter within this field
4) Scroll to the bottom, and enter your choice of number of epochs and number of periods 
5) to see the simulation results at the bottom
'''

#importing dependencies
import numpy as np
import random

#defining the number of epochs and periods based on user input
while True:
    number_epochs = input("Enter the number of epochs (1-50) you would like to model: ")
    try:
       number_epochs = int(number_epochs)
    except ValueError:
       print ('Please enter a valid number between 1 and 50')
       continue
    if 0 <= number_epochs <= 51:
       break
    else:
       print ('Please enter a valid number between 1 and 50')


while True:
    number_periods = input("Enter the number of periods (1-30) you would like to model: ")
    try:
       number_periods = int(number_periods)
    except ValueError:
       print ('Please enter a valid number between 1 and 30')
       continue
    if 0 <= number_periods <= 31:
       break
    else:
       print ('Please enter a valid number between 1 and 30')

#initializing all global variables
#initializing variable that counts the state periods
period = 0 
number_of_periods = number_periods+2 #based on user input
epoch = 1 #initializing variable that counts the epochs
number_of_epochs = number_epochs #based on user input
number_of_agents = 3

#for graphical purposes; showing the action space in the first row of the Q-table
action_count = ("     -0.2 -0.16 -0.12 -0.08 -0.04  0    0.04  0.08  0.12  0.16  0.2") 

global_state = 4.97 #current CO2 emissions are at 4.97 metric tons per capita world-wide. Source: World Bank

#creating a list to store the global state for each epoch
global_state_per_epoch = np.zeros((number_of_epochs, 2)) 

#number of different actions that a single agent can take in a given state
size_of_action_space = 11 

#action space contains of reducing or increasing the amount of tons CO2 consumed per capita
action_space = [-0.2,-0.16,-0.12,-0.08,-0.04,0,0.04,0.08,0.12,0.16,0.2] 
action_index = [0,1,2,3,4,5,6,7,8,9,10]
cost_of_action = 5 #defining cost to reduce CO2 emissions per metric ton

#initializing the Q-tables
Q_LT = np.zeros((number_of_periods-1, size_of_action_space)) 
Q_MT = np.zeros((number_of_periods-1, size_of_action_space)) 
Q_ST = np.zeros((number_of_periods-1, size_of_action_space)) 

#defining best actions from Q-tables
Max_Q_LT = np.argmax(Q_LT[period, :])
Max_Q_MT = np.argmax(Q_MT[period, :])
Max_Q_ST = np.argmax(Q_ST[period, :])

#further aid variable for analysis of cumulative rewards
Q_LT_per_epoch = np.zeros((number_of_epochs, number_of_periods-1))
Q_MT_per_epoch = np.zeros((number_of_epochs, number_of_periods-1))
Q_ST_per_epoch = np.zeros((number_of_epochs, number_of_periods-1))

#defining the weight factors of immediate rewards 
LT_reward_factor = 0.4
MT_reward_factor = 0.5
ST_reward_factor = 0.6 
cumulative_reward = 0 #initializing cumulative reward, which is 0 to start with

#creating a list to store the cumulative reward for each epoch
cumulative_reward_per_epoch = np.zeros((number_of_epochs, 2)) 

#creating a list to store the immediate rewards for each epoch
immediate_rewards_per_epoch = np.zeros((number_of_epochs, number_of_agents + 1)) 

LT_epsilon = 0.9 #epsilon for LT, variable for exploration vs. exploitation, higher epsilon = more exploitation
LT_epsilon_min = 0.1 #defining minimal epsilon for LT
LT_epsilon_decay = 0.95 #defining decay rate of LT's epsilon
MT_epsilon = 0.8
MT_epsilon_min = 0.06
MT_epsilon_decay = 0.9
ST_epsilon = 0.7 
ST_epsilon_min = 0.03 
ST_epsilon_decay = 0.85 
alpha = 0.1 #initializing the learning rate of the Q-values
alpha_min = 0.01 #initializing minimal learning rate after decay
alpha_decay = 0.995 #initializing decay of learning rate
gamma = 0.7  #initializing the reward discount; the higher gamma, the more value a future state creates

#showing period numbers in first column of Q-table
def Q_rowcount(Q_Table):
  subcount = 1
  for row in Q_Table:
    count = f"p{subcount}"
    print(count, row)
    subcount += 1
    

#run epoch loop
for epoch in range (1, number_of_epochs):
  
  #initializing variables which are reset for every new epoch
  global_state = 4.97 #current CO2 emissions are at 4.97 metric tons per capita world-wide. Source: World Bank
  cumulative_reward = 0 #initializing cumulative reward, which is 0 to start with
  alpha = 0.1 #initializing the learning rate of the Q-values
  
  #initializing epsilons for each agent, the variable to balance exploration and exploitation, higher epsilon means more exploitation
  LT_epsilon = 0.9 
  MT_epsilon = 0.8 
  ST_epsilon = 0.7 
  
  #run period loop
  for period in range (0, number_of_periods-2):
    
    #initializing variables
    random_action = random.choice(action_index)
    LT_action = action_space[random_action]
    random_action = random.choice(action_index)
    MT_action = action_space[random_action]
    random_action = random.choice(action_index)
    ST_action = action_space[random_action]
    
    #Immediate Reward functions
    LT_immediate_reward = -1 * LT_action * LT_reward_factor + cost_of_action * LT_action 
    MT_immediate_reward = -1 * MT_action * MT_reward_factor + cost_of_action * MT_action 
    ST_immediate_reward = -1 * ST_action * ST_reward_factor + cost_of_action * ST_action    
    
    #action iteration and Q-Table updates with exploration vs. exploitation
    #LT
    if np.random.rand() <= LT_epsilon:
      random_action = random.choice(action_index)
      LT_action = action_space[random_action]
      Q_LT[period, random_action] = round((1-alpha) * Q_LT[period, random_action] + alpha * (LT_immediate_reward + gamma * np.amax(Q_LT[period + 1, :])), 2)
    else:
      LT_action = action_space[Max_Q_LT]
      Q_LT[period, Max_Q_LT] = round((1-alpha) * Q_LT[period, Max_Q_LT] + alpha * (LT_immediate_reward + gamma * np.amax(Q_LT[period + 1, :])), 2)
    
    #MT
    if np.random.rand() <= MT_epsilon:
      random_action = random.choice(action_index)
      MT_action = action_space[random_action]
      Q_MT[period, random_action] = round((1-alpha) * Q_MT[period, random_action] + alpha * (MT_immediate_reward + gamma * np.amax(Q_MT[period + 1, :])), 2)
    else:
      MT_action = action_space[Max_Q_MT]
      Q_MT[period, Max_Q_MT] = round((1-alpha) * Q_MT[period, Max_Q_MT] + alpha * (LT_immediate_reward + gamma * np.amax(Q_MT[period + 1, :])), 2)
    
    #ST
    if np.random.rand() <= ST_epsilon:
      random_action = random.choice(action_index)
      ST_action = action_space[random_action]
      Q_ST[period, random_action] = round((1-alpha) * Q_ST[period, random_action] + alpha * (ST_immediate_reward + gamma * np.amax(Q_ST[period + 1, :])), 2)
    else:
      ST_action = action_space[Max_Q_ST]
      Q_ST[period, Max_Q_ST] = round((1-alpha) * Q_ST[period, Max_Q_ST] + alpha * (ST_immediate_reward + gamma * np.amax(Q_ST[period + 1, :])), 2)
    
    #to be maximized: += immediate rewards of both agents + the negative of the actions that each agent takes, because the actions influence the global state, which shall be minimized
    cumulative_reward += LT_immediate_reward + MT_immediate_reward + ST_immediate_reward - 5*(action_space[np.argmax(Q_LT[period, :])]) - 5*(action_space[np.argmax(Q_MT[period, :])]) - 5*(action_space[np.argmax(Q_ST[period, :])])
    
    #decaying learning rate and agent's epsilon values
    if alpha > alpha_min:
      alpha *= alpha_decay
    else:
      alpha = alpha
    if LT_epsilon > LT_epsilon_min:
      LT_epsilon *= LT_epsilon_decay
    else:
      LT_epsilon = LT_epsilon
    if MT_epsilon > MT_epsilon_min:
      MT_epsilon *= MT_epsilon_decay
    else:
      MT_epsilon = MT_epsilon
    if ST_epsilon > ST_epsilon_min:
      ST_epsilon *= ST_epsilon_decay
    else:
      ST_epsilon = ST_epsilon
    
    #udpating global state and period counter
    global_state += (LT_action + MT_action + ST_action) / number_of_agents
    period +=1
  
  #for each epoch, store the policies of each agent
  Q_LT_per_epoch[epoch, :] = np.vstack([np.argmax(Q_LT, axis=1).tolist()])
  Q_MT_per_epoch[epoch, :] = np.vstack([np.argmax(Q_MT, axis=1).tolist()])
  Q_ST_per_epoch[epoch, :] = np.vstack([np.argmax(Q_ST, axis=1).tolist()])
  
  #store the cumulative and immediate rewards and the global states of each epoch
  cumulative_reward_per_epoch[epoch, :] = (epoch, cumulative_reward)
  immediate_rewards_per_epoch[epoch, :] = (epoch, LT_immediate_reward, MT_immediate_reward, ST_immediate_reward)
  global_state_per_epoch[epoch, :] = (epoch, global_state)
  
  epoch += 1
  
  

#evaluating the trained model
global_state_per_epoch[0][1] = 4.97
print('\n')
print(f"Best Epoch based on Cumulative Reward: {np.argmax(cumulative_reward_per_epoch[:, 1])}")
print(f"Highest Cumulative Reward: {round(np.amax(cumulative_reward_per_epoch[:, 1]), 2)}")
print(f"Best Epoch based on Global State: {np.argmin(global_state_per_epoch[:, 1])}")
print(f"Lowest Global State: {round(np.min(global_state_per_epoch[:, 1]), 2)}")
print('\n')
print(f"Best Epoch based on LT's Immediate Reward: {np.argmax(immediate_rewards_per_epoch[:, 1])}")
print(f"Highest Immediate Reward for LT: {round(np.max(immediate_rewards_per_epoch[:, 1]), 2)}")
print(f"Best Epoch based on MT's Immediate Reward: {np.argmax(immediate_rewards_per_epoch[:, 2])}")
print(f"Highest Immediate Reward for MT: {round(np.max(immediate_rewards_per_epoch[:, 2]), 2)}")
print(f"Best Epoch based on ST's Immediate Reward: {np.argmax(immediate_rewards_per_epoch[:, 3])}")
print(f"Highest Immediate Reward for ST: {round(np.max(immediate_rewards_per_epoch[:, 3]), 2)}")

print('\n')

#build utilitarian, selfish and greedy policies of each agent
Q_LT_Best = Q_LT_per_epoch[np.argmax(cumulative_reward_per_epoch[:, 1]).tolist()]
Q_MT_Best = Q_MT_per_epoch[np.argmax(cumulative_reward_per_epoch[:, 1]).tolist()]
Q_ST_Best = Q_ST_per_epoch[np.argmax(cumulative_reward_per_epoch[:, 1]).tolist()]

Q_LT_Immediate_Best = Q_LT_per_epoch[np.argmax(immediate_rewards_per_epoch[:, 1]).tolist()]
Q_MT_Immediate_Best = Q_MT_per_epoch[np.argmax(immediate_rewards_per_epoch[:, 1]).tolist()]
Q_ST_Immediate_Best = Q_ST_per_epoch[np.argmax(immediate_rewards_per_epoch[:, 2]).tolist()]
                                                       
LT_Strategy = [action_space[i] for i in Q_LT_Best.astype(int)[1:]]
MT_Strategy = [action_space[i] for i in Q_MT_Best.astype(int)[1:]]
ST_Strategy = [action_space[i] for i in Q_ST_Best.astype(int)[1:]]

LT_Greedy_Strategy = [action_space[i] for i in Q_LT_Immediate_Best.astype(int)[1:]]
MT_Greedy_Strategy = [action_space[i] for i in Q_MT_Immediate_Best.astype(int)[1:]]
ST_Greedy_Strategy = [action_space[i] for i in Q_ST_Immediate_Best.astype(int)[1:]]

LT_Policy = [action_space[i] for i in np.argmax(Q_LT, axis=1)]
MT_Policy = [action_space[i] for i in np.argmax(Q_MT, axis=1)]
ST_Policy = [action_space[i] for i in np.argmax(Q_ST, axis=1)]

print(f"LT's Strategy to achieve Highest Cumulative Reward: \n {LT_Strategy}")
print(f"LT's Strategy to achieve Highest Immediate Reward: \n {LT_Greedy_Strategy}")
print(f"Selfish Policy of LT, based on LT's Final Q-Table: \n {LT_Policy}")
print('\n')

print(f"MT's Strategy to achieve Highest Cumulative Reward: \n {MT_Strategy}")
print(f"MT's Strategy to achieve Highest Immediate Reward: \n {MT_Greedy_Strategy}")
print(f"Selfish Policy of MT, based on MT's Final Q-Table: \n {MT_Policy}")
print('\n')

print(f"ST's Strategy to achieve Highest Cumulative Reward: \n {ST_Strategy}")
print(f"ST's Strategy to achieve Highest Immediate Reward: \n {ST_Greedy_Strategy}")
print(f"Selfish Policy of ST, based on ST's Final Q-Table: \n {ST_Policy}")
print('\n')

print(f"Final Q-Table of LT: \n {action_count}") #Q-Tables are printed...
print(Q_rowcount(Q_LT[:number_of_periods-2]))
print('\n')

print(f"Final Q-Table of MT: \n {action_count}") #Q-Tables are printed...
print(Q_rowcount(Q_MT[:number_of_periods-2]))
print('\n')

print(f"Final Q-Table of ST: \n {action_count}") #...Based on the "rewards_per_epoch" Table, the best Q-Tables are identified as policies
print(Q_rowcount(Q_ST[:number_of_periods-2]))



#Under Selfish Policies
#initializing variables which are reset for every new epoch
global_state = 4.97 #current CO2 emissions are at 4.97 metric tons per capita world-wide. Source: World Bank
cumulative_reward_selfish = 0 #initializing cumulative reward, which is 0 to start with
alpha = 0.1 #initializing the learning rate of the Q-values
Q_LT = np.zeros((number_of_periods-1, size_of_action_space))
Q_MT = np.zeros((number_of_periods-1, size_of_action_space)) 
Q_ST = np.zeros((number_of_periods-1, size_of_action_space))
  
#run period loop
for period in range (0, number_of_periods-2):
  
  #actions based on Policies
  LT_action = LT_Policy[period]
  LT_action = MT_Policy[period]
  ST_action = ST_Policy[period]

  #Immediate Reward functions
  LT_immediate_reward = -1 * LT_action * LT_reward_factor + cost_of_action * LT_action #defining immediate reward function of LT per period
  MT_immediate_reward = -1 * LT_action * MT_reward_factor + cost_of_action * MT_action #defining immediate reward function of LT per period
  ST_immediate_reward = -1 * ST_action * ST_reward_factor + cost_of_action * ST_action #defining immediate reward function of ST per period    

  #to be maximized: += immediate rewards of both agents + the negative of the actions that each agent takes, because the actions influence the global state, which shall be minimized
  cumulative_reward_selfish += LT_immediate_reward + MT_immediate_reward + ST_immediate_reward - 5*(action_space[np.argmax(Q_LT[period, :])]) - 5*(action_space[np.argmax(Q_MT[period, :])]) - 5*(action_space[np.argmax(Q_ST[period, :])])
  
  global_state += (LT_action + MT_action + ST_action) / number_of_agents
  period +=1

print('\n')
print(f"Cumulative Reward under Selfish Policies: {round(cumulative_reward_selfish, 2)}")
print(f"Global State after using Selfish Policies: {round(global_state, 2)}")

Selfish_Reward_Loss = round(100*(cumulative_reward_selfish - np.max(cumulative_reward_per_epoch[:, 1])) / np.max(cumulative_reward_per_epoch[:, 1]),2)
CO2_Selfish = round(100*((global_state - np.min(global_state_per_epoch[:, 1])) / np.min(global_state_per_epoch[:, 1])), 2)

print('\n')
print(f"Percentage of Lower Cumulative Reward comparing Selfish Policies to Best Epoch: {Selfish_Reward_Loss}%")
print(f"Percentage of Higher Global State comparing Selfish Policies to Best Epoch: {CO2_Selfish}%")



#Under Greedy Policies
#initializing variables which are reset for every new epoch
global_state = 4.97 #current CO2 emissions are at 4.97 metric tons per capita world-wide. Source: World Bank
cumulative_reward_greedy = 0 #initializing cumulative reward, which is 0 to start with
alpha = 0.1 #initializing the learning rate of the Q-values
  
#run period loop
for period in range (0, number_of_periods-2):
  
  #actions based on Policies
  LT_action = LT_Greedy_Strategy[period]
  MT_action = MT_Greedy_Strategy[period]
  ST_action = ST_Greedy_Strategy[period]

  #Immediate Reward functions
  LT_immediate_reward = -1 * LT_action * LT_reward_factor + cost_of_action * LT_action 
  MT_immediate_reward = -1 * LT_action * MT_reward_factor + cost_of_action * MT_action 
  ST_immediate_reward = -1 * ST_action * ST_reward_factor + cost_of_action * ST_action    

  #to be maximized: += immediate rewards of both agents + the negative of the actions that each agent takes, because the actions influence the global state, which shall be minimized
  cumulative_reward_greedy += LT_immediate_reward + MT_immediate_reward  + ST_immediate_reward - 5*(action_space[np.argmax(Q_LT[period, :])]) - 5*(action_space[np.argmax(Q_MT[period, :])]) - 5*(action_space[np.argmax(Q_ST[period, :])])
  
  global_state += (LT_action + MT_action + ST_action) / number_of_agents
  period +=1

print('\n')
print(f"Cumulative Reward under Greedy Policies: {round(cumulative_reward_greedy, 2)}")
print(f"Global State after using Greedy Policies: {round(global_state, 2)}")

Greedy_Reward_Loss = round(100*(cumulative_reward_greedy - np.max(cumulative_reward_per_epoch[:, 1])) / np.max(cumulative_reward_per_epoch[:, 1]),2)
CO2_Greedy = round(100*((global_state - np.min(global_state_per_epoch[:, 1])) / np.min(global_state_per_epoch[:, 1])), 2)

print('\n')
print(f"Percentage of Lower Cumulative Reward comparing Greedy Policies to Best Epoch: {Greedy_Reward_Loss}%")
print(f"Percentage of Higher Global State comparing Greedy Policies to Best Epoch: {CO2_Greedy}%")
